# 📈 Week 3-3: 회귀 알고리즘 대결!

## 🎯 오늘의 미션
**분류에서 회귀로!** 이번엔 **숫자를 예측**해보자!

### 🔄 분류 vs 회귀 차이점
- **분류**: "이 꽃은 setosa야!" (카테고리)
- **회귀**: "이 집값은 50만달러야!" (연속된 숫자)

### 🥊 오늘의 대결 선수들
1. **Linear Regression** 🏃‍♂️ (기본형 - 단순하고 빠름)
2. **Ridge Regression** 🛡️ (정규화형 - 과적합 방지)
3. **Lasso Regression** ✂️ (피처 선택형 - 불필요한 것 제거)

---


## 🏠 문제 정의: 보스턴 집값 예측

### 🤔 잠깐! 분류 vs 회귀가 뭐가 다른거야?

**02번에서 배운 분류 알고리즘**:
- 🌸 "이 꽃은 setosa야! versicolor야! virginica야!" 
- 📋 **정답이 카테고리** (A, B, C 중에 선택)
- 🎯 예: 스팸메일 판별 (스팸 or 정상), 동물 분류 (개, 고양이, 새)

**03번에서 배울 회귀 알고리즘**:
- 💰 "이 집값은 정확히 45.7만 달러야!"
- 📊 **정답이 연속된 숫자** (무한히 많은 가능성)
- 🎯 예: 집값 예측, 주식 가격, 온도 예측

### 🏠 오늘의 미션: 집값 예측
**상황**: 부동산 중개업자가 되어 집값을 예측해보자!

**입력 데이터 (X)**:
- 방 개수, 범죄율, 교통 접근성 등 13가지 정보

**예측 목표 (y)**:
- 집값 (단위: 천 달러) - 23.5만, 31.2만, 45.7만... 등등

**왜 회귀인가?**
- 집값은 연속된 숫자 (23.5만, 31.2만, 45.7만...)
- "비싸다/싸다" 같은 카테고리가 아니라 **정확한 숫자**를 맞춰야 함!


In [30]:
# 🛠️ 도구 준비
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

print("🔧 회귀 알고리즘 도구 준비 완료!")
print("📊 오늘의 무기들:")
print("   🏃‍♂️ LinearRegression - 기본형")
print("   🛡️ Ridge - 정규화형")
print("   ✂️ Lasso - 피처 선택형")
print("   📏 mean_squared_error - 오차 측정")
print("   🎯 r2_score - 설명력 측정")


🔧 회귀 알고리즘 도구 준비 완료!
📊 오늘의 무기들:
   🏃‍♂️ LinearRegression - 기본형
   🛡️ Ridge - 정규화형
   ✂️ Lasso - 피처 선택형
   📏 mean_squared_error - 오차 측정
   🎯 r2_score - 설명력 측정


## 📊 데이터 탐험: 보스턴 집값 데이터셋

**보스턴 집값 데이터**는 회귀 학습의 **아이리스**와 같은 존재!
- 506개 집 정보
- 13개 특성 (방 개수, 범죄율 등)
- 실제 집값 데이터


In [31]:
# 🏠 보스턴 집값 데이터 로드
# 주의: sklearn 최신 버전에서는 보스턴 데이터가 제거됨
# 대신 캘리포니아 집값 데이터 사용
try:
    # 보스턴 데이터 시도
    from sklearn.datasets import load_boston
    boston = load_boston()
    X, y = boston.data, boston.target
    feature_names = boston.feature_names
    dataset_name = "보스턴"
    print("🏠 보스턴 집값 데이터 로드 성공!")
except:
    # 캘리포니아 데이터 사용
    california = datasets.fetch_california_housing()
    X, y = california.data, california.target
    feature_names = california.feature_names
    dataset_name = "캘리포니아"
    print("🌴 캘리포니아 집값 데이터 로드 성공!")

print(f"\n📋 {dataset_name} 집값 데이터 정보:")
print(f"   🏘️ 총 집 개수: {X.shape[0]:,}개")
print(f"   📊 특성 개수: {X.shape[1]}개")
print(f"   💰 집값 범위: ${y.min():.1f}만 ~ ${y.max():.1f}만")
print(f"   💵 평균 집값: ${y.mean():.1f}만")


🌴 캘리포니아 집값 데이터 로드 성공!

📋 캘리포니아 집값 데이터 정보:
   🏘️ 총 집 개수: 20,640개
   📊 특성 개수: 8개
   💰 집값 범위: $0.1만 ~ $5.0만
   💵 평균 집값: $2.1만


In [32]:
# 📊 데이터 미리보기
df = pd.DataFrame(X, columns=feature_names)
df['집값'] = y

print("🔍 데이터 미리보기 (처음 5개 집):")
print(df.head())

print("\n📈 집값 분포:")
print(f"   최저가: ${y.min():.1f}만")
print(f"   최고가: ${y.max():.1f}만")
print(f"   중간값: ${np.median(y):.1f}만")
print(f"   평균값: ${y.mean():.1f}만")


🔍 데이터 미리보기 (처음 5개 집):
   MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1  8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86   
2  7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85   
3  5.6431      52.0  5.817352   1.073059       558.0  2.547945     37.85   
4  3.8462      52.0  6.281853   1.081081       565.0  2.181467     37.85   

   Longitude     집값  
0    -122.23  4.526  
1    -122.22  3.585  
2    -122.24  3.521  
3    -122.25  3.413  
4    -122.25  3.422  

📈 집값 분포:
   최저가: $0.1만
   최고가: $5.0만
   중간값: $1.8만
   평균값: $2.1만


## 🔪 데이터 준비: 훈련용 vs 테스트용

**분류와 똑같은 과정!**
- X (특성): 방 개수, 범죄율 등
- y (목표): 집값
- 80% 훈련, 20% 테스트


In [33]:
# ✂️ 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42
)

print("✂️ 데이터 분할 완료!")
print(f"   📚 훈련용: {X_train.shape[0]}개 집")
print(f"   🧪 테스트용: {X_test.shape[0]}개 집")
print(f"   💰 훈련용 평균 집값: ${y_train.mean():.1f}만")
print(f"   💰 테스트용 평균 집값: ${y_test.mean():.1f}만")

print("\n🎯 목표: 테스트용 집들의 가격을 정확히 맞춰보자!")


✂️ 데이터 분할 완료!
   📚 훈련용: 16512개 집
   🧪 테스트용: 4128개 집
   💰 훈련용 평균 집값: $2.1만
   💰 테스트용 평균 집값: $2.1만

🎯 목표: 테스트용 집들의 가격을 정확히 맞춰보자!


## 🥊 회귀 알고리즘 3형제 대결!

### 🏃‍♂️ 1라운드: Linear Regression (기본형)

**🤔 Linear Regression이 뭐야?**
- **Linear** = 직선 (일차함수 y = ax + b 기억나죠?)
- 데이터들 사이에 **가장 잘 맞는 직선**을 그어서 예측하는 방법
- 예: 방 개수가 많을수록 집값이 비싸다 (직선 관계)

**특징**: 가장 단순하고 빠른 회귀 알고리즘
**장점**: 해석하기 쉽고 빠름 (수학 공식이 간단)
**단점**: 너무 단순해서 복잡한 패턴 못 잡을 수 있음


In [34]:
# 🏃‍♂️ Linear Regression 훈련
linear_reg = LinearRegression()
linear_reg.fit(X_train, y_train)

# 예측
y_pred_linear = linear_reg.predict(X_test)

# 성능 측정
mse_linear = mean_squared_error(y_test, y_pred_linear)
r2_linear = r2_score(y_test, y_pred_linear)

print("🏃‍♂️ Linear Regression 결과:")
print(f"   📏 MSE (평균제곱오차): {mse_linear:.2f}")
print(f"   🎯 R² Score (설명력): {r2_linear:.3f} ({r2_linear*100:.1f}%)")
print(f"   💡 해석: 실제 집값의 {r2_linear*100:.1f}%를 설명할 수 있음")


🏃‍♂️ Linear Regression 결과:
   📏 MSE (평균제곱오차): 0.56
   🎯 R² Score (설명력): 0.576 (57.6%)
   💡 해석: 실제 집값의 57.6%를 설명할 수 있음


### 🛡️ 2라운드: Ridge Regression (정규화형)

**🤔 Ridge가 뭐야?**
- Linear의 **업그레이드 버전**
- **과적합 방지 기능** 추가 (훈련 데이터에만 너무 맞춰지는 것 방지)
- 마치 **안전벨트** 같은 역할 - 너무 극단적으로 가지 않게 제어

**특징**: 과적합을 방지하는 정규화 기법 사용
**장점**: 안정적이고 과적합 방지 (새로운 데이터에도 잘 맞음)
**단점**: Linear보다 약간 복잡함


In [35]:
# 🛡️ Ridge Regression 훈련
ridge_reg = Ridge(alpha=1.0, random_state=42)
ridge_reg.fit(X_train, y_train)

# 예측
y_pred_ridge = ridge_reg.predict(X_test)

# 성능 측정
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)

print("🛡️ Ridge Regression 결과:")
print(f"   📏 MSE (평균제곱오차): {mse_ridge:.2f}")
print(f"   🎯 R² Score (설명력): {r2_ridge:.3f} ({r2_ridge*100:.1f}%)")
print(f"   💡 해석: 실제 집값의 {r2_ridge*100:.1f}%를 설명할 수 있음")


🛡️ Ridge Regression 결과:
   📏 MSE (평균제곱오차): 0.56
   🎯 R² Score (설명력): 0.576 (57.6%)
   💡 해석: 실제 집값의 57.6%를 설명할 수 있음


### ✂️ 3라운드: Lasso Regression (피처 선택형)

**🤔 Lasso가 뭐야?**
- **자동 정리 기능**이 있는 회귀 알고리즘
- 13개 특성 중에서 **진짜 중요한 것만** 골라서 사용
- 마치 **정리의 여왕** - 필요없는 건 과감히 버림!
- 예: "범죄율은 중요하지만, 이 특성은 별로 안 중요하네" → 0으로 만들어버림

**특징**: 불필요한 특성을 자동으로 제거
**장점**: 중요한 특성만 선택, 해석 용이 (어떤 게 중요한지 알 수 있음)
**단점**: 가끔 중요한 특성도 실수로 제거할 수 있음


In [36]:
# ✂️ Lasso Regression 훈련
lasso_reg = Lasso(alpha=0.1, random_state=42)
lasso_reg.fit(X_train, y_train)

# 예측
y_pred_lasso = lasso_reg.predict(X_test)

# 성능 측정
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
r2_lasso = r2_score(y_test, y_pred_lasso)

print("✂️ Lasso Regression 결과:")
print(f"   📏 MSE (평균제곱오차): {mse_lasso:.2f}")
print(f"   🎯 R² Score (설명력): {r2_lasso:.3f} ({r2_lasso*100:.1f}%)")
print(f"   💡 해석: 실제 집값의 {r2_lasso*100:.1f}%를 설명할 수 있음")

# Lasso의 특별 기능: 피처 선택
selected_features = np.sum(lasso_reg.coef_ != 0)
print(f"   ✂️ 선택된 특성: {selected_features}개 (총 {X.shape[1]}개 중)")


✂️ Lasso Regression 결과:
   📏 MSE (평균제곱오차): 0.61
   🎯 R² Score (설명력): 0.532 (53.2%)
   💡 해석: 실제 집값의 53.2%를 설명할 수 있음
   ✂️ 선택된 특성: 6개 (총 8개 중)


## 🏆 최종 대결 결과 발표!

**🤔 성능 지표가 뭐야? (분류와 다름!)**

**분류에서는**: 정답률 (100개 중 90개 맞춤 = 90%)

**회귀에서는**: 
- **MSE (평균제곱오차)**: 예측이 얼마나 틀렸는지 (낮을수록 좋음)
  - 예: 실제 50만원인데 45만원 예측 → 오차 5만원
- **R² Score (결정계수)**: 얼마나 잘 설명하는지 (높을수록 좋음, 최대 1.0)
  - 0.8 = 80% 설명력 (꽤 좋음)
  - 0.5 = 50% 설명력 (그럭저럭)
  - 0.9+ = 90%+ 설명력 (매우 좋음)


In [37]:
# 🏆 최종 결과 비교
results = {
    '알고리즘': ['🏃‍♂️ Linear', '🛡️ Ridge', '✂️ Lasso'],
    'MSE': [mse_linear, mse_ridge, mse_lasso],
    'R² Score': [r2_linear, r2_ridge, r2_lasso],
    '설명력(%)': [f"{r2_linear*100:.1f}%", f"{r2_ridge*100:.1f}%", f"{r2_lasso*100:.1f}%"]
}

results_df = pd.DataFrame(results)
print("🏆 회귀 알고리즘 대결 최종 결과:")
print("="*50)
print(results_df.to_string(index=False))

# 우승자 결정 (R² Score 기준)
r2_scores = [r2_linear, r2_ridge, r2_lasso]
algorithms = ['Linear 🏃‍♂️', 'Ridge 🛡️', 'Lasso ✂️']

best_idx = np.argmax(r2_scores)
best_algorithm = algorithms[best_idx]
best_r2 = r2_scores[best_idx]

print(f"\n🥇 우승자: {best_algorithm}")
print(f"🎯 최고 성능: {best_r2:.3f} ({best_r2*100:.1f}% 설명력)")

# 성능 차이 분석
print("\n📊 성능 분석:")
if abs(r2_linear - r2_ridge) < 0.01:
    print("   • Linear와 Ridge 성능이 비슷함 → 과적합 위험 낮음")
if r2_lasso < max(r2_linear, r2_ridge) - 0.02:
    print("   • Lasso 성능 저하 → 중요한 특성을 제거했을 가능성")
else:
    print("   • 모든 알고리즘이 비슷한 성능 → 데이터가 잘 정제됨")


🏆 회귀 알고리즘 대결 최종 결과:
       알고리즘      MSE  R² Score 설명력(%)
🏃‍♂️ Linear 0.555892  0.575788  57.6%
   🛡️ Ridge 0.555803  0.575855  57.6%
   ✂️ Lasso 0.613512  0.531817  53.2%

🥇 우승자: Ridge 🛡️
🎯 최고 성능: 0.576 (57.6% 설명력)

📊 성능 분석:
   • Linear와 Ridge 성능이 비슷함 → 과적합 위험 낮음
   • Lasso 성능 저하 → 중요한 특성을 제거했을 가능성


## 🔍 실제 예측 테스트: 새로운 집 가격 맞춰보기!

**실전 테스트**: 우승 알고리즘으로 실제 집 3개의 가격을 예측해보자!


In [38]:
# 🏠 실제 예측 테스트
# 테스트 데이터에서 3개 집 선택
test_indices = [0, 1, 2]
sample_houses = X_test[test_indices]
actual_prices = y_test[test_indices]

print("🏠 실제 집 3개 가격 예측 대결!")
print("="*60)

for i, idx in enumerate(test_indices):
    house = sample_houses[i:i+1]  # 2D 배열로 만들기
    actual = actual_prices[i]
    
    # 각 알고리즘으로 예측
    pred_linear = linear_reg.predict(house)[0]
    pred_ridge = ridge_reg.predict(house)[0]
    pred_lasso = lasso_reg.predict(house)[0]
    
    print(f"\n🏡 집 #{i+1}:")
    print(f"   💰 실제 가격: ${actual:.1f}만")
    print(f"   🏃‍♂️ Linear 예측: ${pred_linear:.1f}만 (오차: ${abs(actual-pred_linear):.1f}만)")
    print(f"   🛡️ Ridge 예측: ${pred_ridge:.1f}만 (오차: ${abs(actual-pred_ridge):.1f}만)")
    print(f"   ✂️ Lasso 예측: ${pred_lasso:.1f}만 (오차: ${abs(actual-pred_lasso):.1f}만)")
    
    # 가장 정확한 예측 찾기
    errors = [abs(actual-pred_linear), abs(actual-pred_ridge), abs(actual-pred_lasso)]
    best_pred_idx = np.argmin(errors)
    best_pred_name = ['Linear 🏃‍♂️', 'Ridge 🛡️', 'Lasso ✂️'][best_pred_idx]
    print(f"   🎯 이 집 최고 예측: {best_pred_name}")


🏠 실제 집 3개 가격 예측 대결!

🏡 집 #1:
   💰 실제 가격: $0.5만
   🏃‍♂️ Linear 예측: $0.7만 (오차: $0.2만)
   🛡️ Ridge 예측: $0.7만 (오차: $0.2만)
   ✂️ Lasso 예측: $1.0만 (오차: $0.6만)
   🎯 이 집 최고 예측: Linear 🏃‍♂️

🏡 집 #2:
   💰 실제 가격: $0.5만
   🏃‍♂️ Linear 예측: $1.8만 (오차: $1.3만)
   🛡️ Ridge 예측: $1.8만 (오차: $1.3만)
   ✂️ Lasso 예측: $1.6만 (오차: $1.2만)
   🎯 이 집 최고 예측: Lasso ✂️

🏡 집 #3:
   💰 실제 가격: $5.0만
   🏃‍♂️ Linear 예측: $2.7만 (오차: $2.3만)
   🛡️ Ridge 예측: $2.7만 (오차: $2.3만)
   ✂️ Lasso 예측: $2.3만 (오차: $2.7만)
   🎯 이 집 최고 예측: Linear 🏃‍♂️


## 🎓 오늘의 핵심 배운 점

### 🔄 분류 vs 회귀 차이점
- **분류**: 카테고리 예측 (꽃 종류, 스팸 여부)
- **회귀**: 연속된 숫자 예측 (집값, 온도, 주가)

### 🥊 회귀 알고리즘 특성
1. **Linear Regression 🏃‍♂️**
   - 장점: 빠르고 해석 쉬움
   - 단점: 과적합 위험
   - 언제 사용: 단순한 데이터, 빠른 결과 필요시

2. **Ridge Regression 🛡️**
   - 장점: 과적합 방지, 안정적
   - 단점: 약간 복잡
   - 언제 사용: 과적합이 걱정될 때

3. **Lasso Regression ✂️**
   - 장점: 자동 피처 선택, 해석 용이
   - 단점: 중요한 특성도 제거 가능
   - 언제 사용: 특성이 많고 중요한 것만 골라내고 싶을 때

### 📊 성능 지표 이해
- **MSE**: 예측 오차 (낮을수록 좋음)
- **R² Score**: 설명력 (높을수록 좋음, 최대 1.0)

### 🚀 다음 단계
- 교차 검증 (Cross Validation)
- 혼동행렬과 정밀도/재현율
- 첫 ML 프로젝트 (타이타닉!)


## 🎉 축하합니다!

**Week 3 scikit-learn 기초 완전 정복!** 🏆

✅ **완료한 것들**:
- 분류 알고리즘 (Decision Tree, Random Forest, SVM)
- 회귀 알고리즘 (Linear, Ridge, Lasso)
- 모델 평가 방법 (train_test_split)
- 실전 예측 및 성능 분석

**이제 진짜 AI 개발자 입문 완료!** 🎓✨
